In [75]:
import numpy as np
import scipy.stats as st
from scipy.stats import norm

In [77]:
S0 = 100 
K = 100
T = 1  
r = 0.05
sigma = 0.2 
N = 100000

In [79]:
def monte_carlo_option_pricing(S0, K, T, r, sigma, N, option_type="call",confidence_level=0.95):
    stock_prices = []
    np.random.seed(42) 
    Z = np.random.normal(0, 1, N)
    ST = S0 * np.exp((r - 0.5 * sigma**2) * T + sigma * np.sqrt(T) * Z)
    stock_prices.append(ST)
    if option_type == "call":
        payoffs = np.maximum(ST - K, 0)
    elif option_type == "put":
        payoffs = np.maximum(K - ST, 0)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")
    
    discounted_payoffs = np.exp(-r * T) * payoffs
    option_price = np.mean(discounted_payoffs)    
    std_error = np.std(discounted_payoffs, ddof=1) / np.sqrt(N)
    ci_half_width = st.norm.ppf((1 + confidence_level) / 2) * std_error
    confidence_interval = (option_price - ci_half_width, option_price + ci_half_width)
        
    return option_price, confidence_interval

def black_scholes_option_price(S0, K, T, r, sigma, option_type="call"):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    
    if option_type == "call":
        price = S0 * norm.cdf(d1) - K * np.exp(-r * T) * norm.cdf(d2)
    elif option_type == "put":
        price = K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)
    else:
        raise ValueError("Invalid option type. Choose 'call' or 'put'.")
    
    return price

def binomial_tree_option_pricing(S0, K, T, r, sigma, N, option_type="call"):
    dt = T / N
    u = np.exp(sigma * np.sqrt(dt))
    d = 1 / u
    p = (np.exp(r * dt) - d) / (u - d)
    
    price_tree = np.zeros((N + 1, N + 1))
    for j in range(N + 1):
        price_tree[j, N] = max(0, (S0 * (u ** j) * (d ** (N - j)) - K) if option_type == "call" else (K - S0 * (u ** j) * (d ** (N - j))))
    
    for i in range(N - 1, -1, -1):
        for j in range(i + 1):
            price_tree[j, i] = np.exp(-r * dt) * (p * price_tree[j + 1, i + 1] + (1 - p) * price_tree[j, i + 1])
    
    return price_tree[0, 0]

In [85]:
call_price, call_ci = monte_carlo_option_pricing(S0, K, T, r, sigma, N, "call")
put_price, put_ci = monte_carlo_option_pricing(S0, K, T, r, sigma, N, "put")
call_price_bs = black_scholes_option_price(S0, K, T, r, sigma, "call")
put_price_bs = black_scholes_option_price(S0, K, T, r, sigma, "put")
call_price_bt = binomial_tree_option_pricing(S0, K, T, r, sigma, 100, "call")
put_price_bt = binomial_tree_option_pricing(S0, K, T, r, sigma, 100, "put")

In [89]:
print(f"Black-Scholes Call Option Price: {call_price_bs:.4f}")
print(f"Black-Scholes Put Option Price: {put_price_bs:.4f}")
print(f"Monte-Carlo Call option price: {call_price:.4f}, 95% CI: ({call_ci[0]:.4f}, {call_ci[1]:.4f})")
print(f"Monte-Carlo Put option price: {put_price:.4f}, 95% CI: ({put_ci[0]:.4f}, {put_ci[1]:.4f})")
print(f"Binomial Tree Call Option Price: {call_price_bt:.4f}")
print(f"Binomial Tree Put Option Price: {put_price_bt:.4f}")


Black-Scholes Call Option Price: 10.4506
Black-Scholes Put Option Price: 5.5735
Monte-Carlo Call option price: 10.4739, 95% CI: (10.3826, 10.5652)
Monte-Carlo Put option price: 5.5742, 95% CI: (5.5205, 5.6279)
Binomial Tree Call Option Price: 10.4306
Binomial Tree Put Option Price: 5.5536
